This notebook file is to show the steps we followed to extract parking lots from OpenStreetMap of major diy/wholesale.

In [ ]:
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gpd
import time
from tqdm import tqdm
import sys
import os
from pathlib import Path

path = Path(os.getcwd())
parent_directory = Path.joinpath(path.parent,'src').as_posix()
sys.path.append(str(parent_directory))

import pandas as pd
from tqdm import tqdm

### Extract major diy/wholesale or other stores, Germany

In [ ]:
location_of_interest = 'Germany'
tag_of_interest = 'brand'
subtags_of_interest = [] #select brands of interest
distance = 10

In [ ]:
places = (ox.features_from_place(location_of_interest, 
                                              tags={tag_of_interest: subtags_of_interest})
                       .reset_index())

places['geometry'] = np.where(places['geometry'].geom_type == 'Point', places['geometry'], places['geometry'].centroid)
places['lon'] = places.geometry.apply(lambda p: p.x)
places['lat'] = places.geometry.apply(lambda p: p.y)

##### Extract all parking lots within a distance of 10m around those places

In [ ]:
time_out = abs(np.random.default_rng(1234).normal(2, 3, places.shape[0]))
tags = {'parking':True}
parking_all = gpd.GeoDataFrame()

for place in tqdm(places.osmid.unique().tolist()):
    try:
        tmp = ox.features_from_point(places.loc[places.osmid == place, ['lat', 'lon']].values.flatten(), dist=distance, tags=tags).reset_index()
        tmp['osmid_origin'] = place
        parking_all = pd.concat([parking_all, tmp])
        time.sleep(time_out[places.osmid.unique().tolist().index(place)])  ### to avoid overloading the server    
    except:
        continue

In [ ]:
parking_lots = parking_all.copy()
parking_lots.head(1)

In [ ]:
parking_lots.shape

Just keep surface and rooftop parking

In [ ]:
parking_lots = parking_lots[(parking_lots.parking == 'surface') | (parking_lots.parking == 'rooftop')].copy()

Keep only commercial parking

In [ ]:
parking_lots = parking_lots[(parking_lots.access == 'customers')].copy()

Keep only those that have an area

In [ ]:
parking_poly = parking_lots.loc[parking_lots['geometry'].geom_type == 'Polygon',['osmid', 'osmid_origin', 'geometry']]

In [ ]:
parking_poly['area'] = parking_poly.geometry.area
parking_poly.head()

Calculate distance of parking lot to origin POI

In [ ]:
parking_poly['dist_to_poi'] = (parking_poly
                               .to_crs('EPSG:3857')
                               .merge(places
                                      .to_crs('EPSG:3857')
                                      .rename(columns = {'osmid':'osmid_origin', 'geometry':'geo_point'})[['osmid_origin', 'geo_point']]
                                      , on = 'osmid_origin', how = 'left')
                               .apply(lambda row: row['geometry'].distance(row['geo_point']), axis=1))

Calculate parking lot size

In [ ]:
parking_poly['area_sqm'] = parking_poly.to_crs('EPSG:3857').area

In [ ]:
parking_poly.head()

In [ ]:
parking_poly.info()

##### Save it

In [ ]:
parking_poly.head()

In [ ]:
parking_poly.to_file('../../input/diy_parking_poly.geojson', driver='GeoJSON')